<a href="https://colab.research.google.com/github/RicardoAz15/FAA/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extract dataset files

In [3]:
from zipfile import ZipFile

file_name = 'drive/MyDrive/Colab Notebooks/dataset.zip'

with ZipFile(file_name, 'r') as zip:
  print('Extracting all the files now...')
  zip.extractall()
  print('Done!')

Extracting all the files now...
Done!


Extraction and Analise of data

Import necessary packages

In [4]:
import glob
import random
import numpy as np
import matplotlib.pyplot as plt

Initialize global variables

In [5]:


TRAINING_DATA_LANDMARK_PATH = 'dataset/train_set/*/annotations/*_lnd.npy'
VALIDATION_DATA_LANDMARK_PATH = 'dataset/val_set/*/annotations/*_lnd.npy'

TRAINING_DATA_EXP_PATH = 'dataset/train_set/*/annotations/*_exp.npy'
VALIDATION_DATA_EXP_PATH = 'dataset/val_set/*/annotations/*_exp.npy'


In [16]:
#LABELS
training_labels_paths = glob.glob(TRAINING_DATA_EXP_PATH)
validation_labels_paths = glob.glob(VALIDATION_DATA_EXP_PATH)

#LANDMAKRS
training_landmarks_paths = glob.glob(TRAINING_DATA_LANDMARK_PATH)
validation_landmarks_paths = glob.glob(VALIDATION_DATA_LANDMARK_PATH)

print(len(training_landmarks_paths))

 

46236


Organize data

In [92]:
def organize_data(labels_paths, landmarks_paths):
  size = len(labels_paths)

  labels = np.empty((0,2))
  landmarks = np.empty((0,68,2))

  for label_file in labels_paths:
    # get index and landmark_file
    file_number = (label_file.split('/')[-1]).split('_')[0]
    landmark_file = [f for f in landmarks_paths if file_number in f][0]

    # get label value and landmarks coordinates
    label_value = np.array(([int(np.load(label_file))]))
    landmark_coordinates = np.load(landmark_file)

    result_landmarks = landmark_coordinates[:,1:]
    
    labels = np.insert(labels,0,label_value)
    landmarks = np.insert(landmarks,0,result_landmarks,axis=0)
    

  return labels, landmarks
 

In [93]:
# training data

training_labels, training_landmarks = organize_data(training_labels_paths, training_landmarks_paths)
# validation_labels, validation_landmarks = organize_data(validation_labels_paths, validation_landmarks_paths)

Plot data

In [94]:
happy = (training_labels==0)
sad = (training_labels==1)

print(training_landmarks.shape)

(10, 68, 2)


Train SVM model

Linear SVM 

In [25]:
from sklearn.svm import SVC

linear_classifier = SVC(C=50, kernel='linear')

print(training_landmarks.shape)
print(training_labels.shape)

linear_classifier.fit(training_landmarks,np.ravel(training_labels) )



(46336, 68, 2)
(92572,)


ValueError: ignored

In [ ]:
from sklearn import metrics

print("Accuracy: " , metrics.accuracy_score(training_landmarks, training_labels))
print("Precision: ", metrics.precision_score(training_landmarks, training_labels))
print("Recall: ", metrics.recall_score(training_landmarks, training_labels))


In [ ]:
# test prediction
predictions = linear_classifier.predict(validation_landmarks)




NonLinear SVM

In [ ]:
def calculateParams(X, y, Xval, yval, vals):
    """
    returns the optimal C and gamma(1/sigma) based on a cross-validation set.
    """
    acc = 0
    best_C=0
    best_gamma=0
    for i in vals:
        C= i
        for j in vals:
            gamma = 1/j
            classifier = SVC(C=C,gamma=gamma)
            classifier.fit(X,y)
            prediction = classifier.predict(Xval)
            score = classifier.score(Xval,yval)
            if score>acc:
                acc =score
                best_C =C
                best_gamma=gamma
    return best_C, best_gamma

RBF SVM